In [1]:
# Service taht simply returns you IP address in a JSON
import requests
r = requests.get('https://api.ipify.org?format=json')
j = r.json()
print(j)


{'ip': '34.230.156.67'}


In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import subprocess

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model


In [3]:
# Load in the data
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test/ 255.0
print("x_train.shape:", x_train.shape)
print("x_test.shape:", x_test.shape)


x_train.shape: (60000, 28, 28)
x_test.shape: (10000, 28, 28)


In [4]:
# the data is only 2D!
# convolution expects height x width x color
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train.shape:", x_train.shape)
print("x_test.shape:", x_test.shape)

x_train.shape: (60000, 28, 28, 1)
x_test.shape: (10000, 28, 28, 1)


In [5]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [6]:
# Build the model using the functional API
i = Input(shape=x_train[0].shape)  # tensorflow.keras.layers.Input
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656

In [7]:
# Compile and fit
# Note: make sure youare using the GPU for this!
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 21s 358us/sample - loss: 0.5223 - accuracy: 0.8066 - val_loss: 0.3976 - val_accuracy: 0.8478
Epoch 2/5
60000/60000 [==============================] - 20s 341us/sample - loss: 0.3609 - accuracy: 0.8650 - val_loss: 0.3444 - val_accuracy: 0.8716
Epoch 3/5
60000/60000 [==============================] - 24s 402us/sample - loss: 0.3109 - accuracy: 0.8835 - val_loss: 0.3376 - val_accuracy: 0.8765
Epoch 4/5
60000/60000 [==============================] - 20s 332us/sample - loss: 0.2769 - accuracy: 0.8948 - val_loss: 0.3142 - val_accuracy: 0.8876
Epoch 5/5
60000/60000 [==============================] - 20s 336us/sample - loss: 0.2532 - accuracy: 0.9054 - val_loss: 0.3027 - val_accuracy: 0.8901


In [8]:
# Save the model to a temporary directory
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}
    
tf.saved_model.save(model, export_path)
# model.save(export_path)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/1/assets

Saved model:
total 168
drwxr-xr-x 2 ec2-user ec2-user   4096 Jan 30 16:19 assets
-rw-rw-r-- 1 ec2-user ec2-user 161936 Jan 30 16:19 saved_model.pb
drwxr-xr-x 2 ec2-user ec2-user   4096 Jan 30 16:19 variables


In [ ]:
#tf.saved_model.contains_saved_model(export_path)


In [ ]:
#!saved_model_cli show --dir {export_path} --all


In [ ]:
#new_model = tf.saved_model.load(export_path)

In [ ]:
#outputs = new_model(x_test[1])

In [ ]:
#f = new_model.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))